<a href="https://colab.research.google.com/github/sajidjavid222/CiSTUP_IISc_python_test/blob/main/question2_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
dataset = pd.read_csv("combined_trajectories.csv")
!pip install geopandas


In [ ]:
import pandas as pd
import numpy as np
from multiprocessing import Pool


def calculate_distance(df, individual_id):
    sub_df = df[df['individual_id'] == individual_id]
    sub_df = sub_df.sort_values(by=['trajectory_id', 'date', 'time'])
    sub_df['prev_lat'] = sub_df['latitude'].shift(1)
    sub_df['prev_long'] = sub_df['longitude'].shift(1)
    sub_df['distance'] = np.sqrt((sub_df['latitude'] - sub_df['prev_lat']) ** 2 +
                                 (sub_df['longitude'] - sub_df['prev_long']) ** 2)
    return sub_df['distance'].sum()


def total_distance_by_individual(df):
    individual_ids = df['individual_id'].unique()
    pool = Pool(processes=len(individual_ids))
    results = [pool.apply_async(calculate_distance, args=(df, i)) for i in individual_ids]
    output = [(individual_ids[i], result.get()) for i, result in enumerate(results)]
    pool.close()
    pool.join()
    return pd.DataFrame(output, columns=['individual_id', 'total_distance']).sort_values('individual_id').reset_index(drop=True)

df = total_distance_by_individual(dataset)
print(df)




In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import seaborn as sns

def extract_and_visualize_hotspots(dataset_path):
    # Load the GPS dataset
    gps_df = pd.read_csv(dataset_path, parse_dates=[['date', 'time']])

    # Remove rows with missing date-time values
    gps_df = gps_df.dropna(subset=['date_time'])

    # Define the boundary of Beijing City as a polygon
    beijing_polygon = Polygon([(116.011933,39.882817),(116.515274,39.872444),(116.56257,40.146702),(116.267143,40.254353),(116.011933,39.882817)])

    # Convert the GPS dataset to a GeoDataFrame
    gps_points = [Point(xy) for xy in zip(gps_df['longitude'], gps_df['latitude'])]
    gps_gdf = gpd.GeoDataFrame(gps_df, geometry=gps_points, crs="EPSG:4326")

    # Filter the GPS data to only contain trips within Beijing City
    gps_gdf = gps_gdf[gps_gdf.within(beijing_polygon)]

    # Visualize the spatial distribution of GPS points
    fig, ax = plt.subplots(figsize=(10, 10))
    gps_gdf.plot(ax=ax, markersize=1, color='red', alpha=0.5)
    beijing_polygon_plot = gpd.GeoDataFrame({'geometry': beijing_polygon}, index=[0], crs="EPSG:4326")
    beijing_polygon_plot.plot(ax=ax, facecolor='none', edgecolor='black')
    ax.set_title('Spatial Distribution of GPS Points in Beijing City')
    plt.show()

    # Extract the temporal hotspots of GPS points
    gps_gdf['datetime'] = pd.to_datetime(gps_gdf['date_time'])
    gps_gdf['dayofweek'] = gps_gdf['datetime'].dt.dayofweek
    gps_gdf['hour'] = gps_gdf['datetime'].dt.hour
    hotspot_counts = gps_gdf.groupby(['dayofweek', 'hour'])['latitude'].count().reset_index(name='count')

    # Visualize the temporal hotspots of GPS points
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(hotspot_counts.pivot('dayofweek', 'hour', 'count'), cmap='YlOrRd', ax=ax)
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Day of Week')
    ax.set_title('Number of GPS Points by Day of Week and Hour in Beijing City')
    plt.show()


extract_and_visualize_hotspots("combined_trajectories.csv")

In [ ]:
!pip install folium --upgrade


In [ ]:
import pandas as pd
import folium
from folium import plugins

# Load the dataset
df = pd.read_csv('combined_trajectories.csv')

# Filter to only contain trips within Beijing City
beijing_polygon = [(39.4, 115.4), (39.4, 117.5), (41.1, 117.5), (41.1, 115.4)]
df = df[(df['latitude'] >= 39.4) & (df['latitude'] <= 41.1) &
        (df['longitude'] >= 115.4) & (df['longitude'] <= 117.5)]
# Group the data by latitude and longitude
heatmap_data = df.groupby(['latitude', 'longitude']).size().reset_index(name='count')
# Create a heatmap using folium
heatmap_map = folium.Map(location=[39.9, 116.4], zoom_start=11)

# Add the heatmap layer to the map
heatmap_map.add_child(folium.plugins.HeatMap(data=heatmap_data[['latitude', 'longitude', 'count']].values,
                                             radius=8, max_val=max(heatmap_data['count'])))

# Display the map
heatmap_map


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# Load the GPS data
gps_data = pd.read_csv('combined_trajectories.csv')

# Drop any rows with missing or invalid values in the latitude and longitude columns
gps_data.dropna(subset=['latitude', 'longitude'], inplace=True)

# Compute the density of GPS points
density = gaussian_kde(gps_data[['latitude', 'longitude']].T)

# Compute the color scale based on the density of GPS points
color_scale = density(gps_data[['latitude', 'longitude']].T)

# Create a scatterplot with a density-based color scale
plt.scatter(gps_data['longitude'], gps_data['latitude'], c=color_scale, s=5)

# Add a colorbar to the plot
cbar = plt.colorbar()
cbar.set_label('Density')

# Set the x and y axis labels
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Set the title of the plot
plt.title('GPS Data Density')

# Show the plot
plt.show()


In [ ]:
import seaborn as sns


# Load the GPS data
gps_data = pd.read_csv('combined_trajectories.csv')

# Generate a contour plot
sns.kdeplot(data=gps_data, x="longitude", y="latitude", fill=True, thresh=0, levels=10, cmap="rocket")


In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d

# Load the GPS data
gps_data = pd.read_csv('combined_trajectories.csv')

# Remove rows with NaN values in longitude or latitude
gps_data = gps_data.dropna(subset=['longitude', 'latitude'])

# Create a GeoDataFrame from the GPS data
geometry = gpd.points_from_xy(gps_data.longitude, gps_data.latitude)
gps_gdf = gpd.GeoDataFrame(gps_data, geometry=geometry)

# Compute the Voronoi diagram
vor = Voronoi(gps_data[['longitude', 'latitude']].values)

# Plot the Voronoi diagram and the GPS points on a map
fig, ax = plt.subplots(figsize=(10, 10))
gps_gdf.plot(ax=ax, color='blue')
voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='orange')
ax.set_title('Voronoi Map of GPS Points')
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the GPS data into a Pandas dataframe
gps_data = pd.read_csv('combined_trajectories.csv')

import numpy as np

# Define the latitude and longitude coordinates of Beijing city
beijing_lat = (39.50, 41.06)
beijing_lon = (115.42, 117.51)

# Filter the GPS data to only contain trips within Beijing city
beijing_data = gps_data[np.logical_and(gps_data['latitude'] >= beijing_lat[0], gps_data['latitude'] <= beijing_lat[1])
                        & np.logical_and(gps_data['longitude'] >= beijing_lon[0], gps_data['longitude'] <= beijing_lon[1])]

# Create a heatmap to show the density or crowding of the GPS data
sns.kdeplot(x=beijing_data['longitude'], y=beijing_data['latitude'], cmap='YlOrRd', fill=True, thresh=0.05, alpha=0.5)


# Set the axis labels and title of the heatmap
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Spatial Hotspots of Beijing City')

# Display the heatmap
plt.show()

# Analyze the temporal distribution of the GPS data to identify the time periods when the most data points have been generated
beijing_data['datetime'] = pd.to_datetime(beijing_data['date'] + ' ' + beijing_data['time'])
beijing_data['hour'] = beijing_data['datetime'].dt.hour

# Create a bar chart to show the hourly distribution of the GPS data
sns.countplot(x='hour', data=beijing_data)

# Set the axis labels and title of the bar chart
plt.xlabel('Hour of Day')
plt.ylabel('Number of GPS Data Points')
plt.title('Temporal Hotspots of Beijing City')

# Display the bar chart
plt.show()



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def create_heatmap(df):
    # Filter out any NaN values
    df = df.dropna(subset=['latitude', 'longitude'])

    # Create a heatmap using seaborn
    sns.kdeplot(data=df[['latitude', 'longitude']], fill=True, shade_lowest=False)


    # Set the axis labels and title
    plt.xlabel('Latitude')
    plt.ylabel('Longitude')
    plt.title('Location Density Heatmap')

    # Display the plot
    plt.show()

create_heatmap(dataset)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def create_scatterplot(df):
    # Create a kernel density estimate plot of the latitude and longitude using seaborn
    sns.kdeplot(data=df[['latitude', 'longitude']], fill=True)

    # Create a scatter plot of the latitude and longitude using matplotlib
    plt.scatter(df['latitude'], df['longitude'], s=2)

    # Set the axis labels and title
    plt.xlabel('Latitude')
    plt.ylabel('Longitude')
    plt.title('Spatial Distribution of Points')

    # Show the plot
    plt.show()
create_scatterplot(dataset)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def create_scatterplot(df):
    # Create a kernel density estimate plot of the latitude and longitude using seaborn
    sns.kdeplot(data=df[['latitude', 'longitude']], shade=True)

    # Create a scatter plot of the latitude and longitude using matplotlib
    plt.scatter(df['latitude'], df['longitude'], s=2, cmap='coolwarm')

    # Set the axis labels and title
    plt.xlabel('Latitude')
    plt.ylabel('Longitude')
    plt.title('Spatial Distribution of Points')

    # Show the plot
    plt.show()
create_scatterplot(dataset)


In [ ]:
!pip install basemap

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the GPS dataset
gps_data = pd.read_csv('combined_trajectories.csv')

# Group the data by individual_id and count the number of GPS points per individual
count_data = gps_data.groupby('individual_id').count()['latitude']

# Plot the bar chart
plt.bar(count_data.index, count_data.values)
plt.xlabel('Individual ID')
plt.ylabel('Number of GPS Points')
plt.title('Density of GPS Points per Individual')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the GPS dataset
gps_data = pd.read_csv('combined_trajectories.csv')

# Plot a scatter plot of latitude and longitude
plt.scatter(gps_data['longitude'], gps_data['latitude'], s=1)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Scatter Plot of GPS Points')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the GPS dataset
gps_data = pd.read_csv('combined_trajectories.csv')

# Group the data by individual and count the number of GPS points per individual
count_data = gps_data.groupby('individual_id').count()['latitude']

# Plot a bar chart of the number of GPS points per individual
count_data.plot(kind='bar')
plt.xlabel('Individual ID')
plt.ylabel('Number of GPS Points')
plt.title('Bar Chart of GPS Points per Individual')
plt.show()
